<a href="https://colab.research.google.com/github/GuoZhuyin/zhuyinasia/blob/main/DeepLearning/modeltest2_TEAM_855_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 隨機挑選訓練集進行測試判斷模型準確率

In [ ]:
import PIL
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torchvision
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [ ]:
classes=['banana', 'bareland', 'carrot', 'corn', 'dragonfruit', 'garlic', 'guava', 'peanut', 'pineapple', 'pumpkin', 'rice', 'soybean', 'sugarcane', 'tomato', 'building', 'mountain', 'sky']
num_classes = len(classes)
print(num_classes)

In [ ]:
from torchvision import models
#model = models.inception_v3(pretrained=False)
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=False)
model.aux_logits = False
model.fc =  nn.Linear(model.fc.in_features, num_classes)
model = nn.DataParallel(model) #cuda設定

In [ ]:
model.load_state_dict(torch.load('../model/InV3_TEAM_855_5.0.pth'))

輸入的模型參數

In [ ]:
model.eval()

In [ ]:
model = model.cuda()

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
class MyDatasetFromImage(Dataset):
    """FarmLand single dataset."""

    def __init__(self, image, transform=None):
        """
        Args:
            image (Image): singe image.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.image = image
        self.transform = transform
        self.boxes=list()

        count = 0
        width, height = image.size   # Get dimensions
        new_width = 512
        new_height = 512
        stride = 32
        for startx in range(width//3, width*2//3-new_width, stride):
          for starty in range(height//2, height*3//4-new_height, stride):
              left = startx
              top = starty
              right = startx + new_width
              bottom = starty + new_height
              self.boxes.append((left, top, right, bottom))
    def __len__(self):
        return len(self.boxes)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
                # Crop the sub image
        left, top, right, bottom = self.boxes[idx]
        im2 = im.crop((left, top, right, bottom))

        if self.transform:
            sample = self.transform(im2)
        else:
            sample = im2
        return sample

In [ ]:
def accumulate_blocks(stats, arr):
    for index in arr:
      if index == 2 or index == 8 or index == 13:
           continue
      stats[index] +=1

In [ ]:
def judge_class(testset, model, batch_size):
    test_dataloader = DataLoader(testset, batch_size=batch_size, shuffle=False)
    stats = [0 for i in range(17)] 
    for images in test_dataloader:
        images = images.cuda()
        outputs = model(images)
        _, dets = torch.max(outputs, 1)
        accumulate_blocks(stats, dets)
    return stats

In [ ]:
class_to_index={'banana': 0, 'bareland': 1, 'building': 2, 'carrot': 3,
 'corn': 4, 'dragonfruit': 5, 'garlic': 6, 'guava': 7, 'mountain': 8,
 'peanut': 9, 'pineapple': 10, 'pumpkin': 11, 'rice': 12, 'sky': 13,
 'soybean': 14, 'sugarcane': 15, 'tomato': 16}
index_to_class = {v: k for k, v in class_to_index.items()}

In [ ]:
import glob
testpath=os.path.join("Test200", "*") #測試資料夾位置
files = glob.glob(testpath)
files.sort()
for fname in files:
  print(fname)

隨機挑選訓練集的測試資料夾

In [ ]:
import pandas as pd
from PIL import Image 
df = pd.DataFrame(columns =['filename', 'landtype'])
batch_size = 64 #GPU調整batch
for i, fname in enumerate(files):
    img_name = os.path.basename(fname)
    im = Image.open(fname)
    testset = MyDatasetFromImage(im, test_transform)
    stats = judge_class(testset, model, batch_size)
    curclass=index_to_class[np.argmax(stats)]
    df.loc[i]=[img_name, curclass]#填答案
    print(f"{img_name}->{curclass}")

In [ ]:
df.to_csv("result.csv", index = False)

#IPO-P: 計算分數

https://aidea-web.tw/topic/93c8c26b-0e96-44bc-9a53-1c96353ad340

評估方式採用Weighted-Precision(WP), 且各類別F1-score需大於0.7

$$WP={\sum_{i=1}^{NumofClass} (Precision_i \times (TP_i + FN_i)) \over TotalImageCount}$$

$$\text{F1-score}= 2 \times {(Precision \times Recall) \over (Precision + Recall) }$$

$$\text{Recall}= {TP \over TP + FN }$$

$$\text{Precision}= {TP \over TP + FP }$$


In [ ]:
len(df)

In [ ]:
df2 = pd.read_csv('test200.csv')

In [ ]:
# 檢查看看一不一樣
for i in range(len(df)):
  s1=df.iloc[i]
  s2=df2.iloc[i]
  if s1.filename!=s2.filename:
    print(f"{s1.filename}!={s2.filename}")

![](https://www.researchgate.net/publication/336402347/figure/fig3/AS:812472659349505@1570719985505/Calculation-of-Precision-Recall-and-Accuracy-in-the-confusion-matrix.ppm)


In [ ]:
T = 0
All =len(df)
for i in range(len(df)):
  s1=df.iloc[i]
  s2=df2.iloc[i]
  if s1.landtype==s2.landtype:
      T += 1
Accuracy = T/All
print(f"Accuracy={Accuracy:.2f}")
     

In [ ]:
landtypes=['banana', 'bareland', 'carrot', 'corn', 'dragonfruit', 'garlic', 'guava', 'peanut', 'pineapple', 'pumpkin', 'rice', 'soybean', 'sugarcane', 'tomato']
print(len(landtypes))

In [ ]:
TP = dict()
FN = dict()
FP = dict()
for lt in landtypes:
  TP[lt] = 0
  FN[lt] = 0
  FP[lt] = 0
for i in range(len(df)):
  s1=df.iloc[i]
  s2=df2.iloc[i]
  if s1.landtype==s2.landtype:
      TP[s1.landtype] +=1
  else:
      FP[s1.landtype] +=1
      FN[s2.landtype] +=1

In [ ]:
for lt in landtypes:
  if TP[lt]+FN[lt] >0:
    recall = TP[lt]/(TP[lt]+FN[lt])
  else:
    recall = 0
  if TP[lt]+FP[lt] >0:
    precision = TP[lt]/(TP[lt]+FP[lt])
  else:
    precision = 0
  f1_score = 2*precision*recall/(precision+recall)
  print(f"{lt:12}:precision={precision:.2f}:recall={recall:.2f}:f1-score={f1_score:.2f}")

In [ ]:
score = 0.0
for lt in landtypes:
  if TP[lt]+FN[lt] >0:
    recall = TP[lt]/(TP[lt]+FN[lt])
  else:
    recall = 0
  if TP[lt]+FP[lt] >0:
    precision = TP[lt]/(TP[lt]+FP[lt])
  else:
    precision = 0
  score += precision*(TP[lt]+FN[lt])

Weighted_Precision = score/All
print(f"Weighted_Precision={Weighted_Precision:.2f}")